## [ Chapter 13 - Semantic Search with Dense Vectors ] 
## Setting up the outdoors dataset

We're going to use the Outdoors dataset for this chapter, and for a very important reason: the vocabulary and contexts in the outdoor question and answer __domain__ already have good coverage in the Transformer models we'll be using.

This is because the datasets that were used to train the model include sources that are likely to have similar subject matter.  Wikipedia was used to train bert-base-uncased (https://huggingface.co/bert-base-uncased#training-data) and, surprise! wikipedia has a section specifically on outdoors content: https://en.wikipedia.org/wiki/Outdoor

This is important, because if the words and their contexts haven't been seen before, the model will be less accurate.

Also, who doesn't enjoy playing around with a new dataset?! Data is search nerd candy.

In [1]:
import sys
sys.path.append("../..")
import json
import html
from aips import get_engine
from IPython.display import display, HTML

engine = get_engine()

## Listing 13.1

### Creating our Collection and Indexing the documents

In [2]:
#outdoors
![ ! -d 'outdoors' ] && git clone --depth=1 https://github.com/ai-powered-search/outdoors.git
! cd outdoors && git pull
! cd outdoors && cat outdoors.tgz.part* > outdoors.tgz
! cd outdoors && mkdir -p '../../../data/outdoors/' && tar -xvf outdoors.tgz -C '../../../data/outdoors/'

Already up to date.
README.md
concepts.pickle
._guesses.csv
guesses.csv
._guesses_all.json
guesses_all.json
outdoors_concepts.pickle
outdoors_embeddings.pickle
._outdoors_golden_answers.csv
outdoors_golden_answers.csv
._outdoors_golden_answers.xlsx
outdoors_golden_answers.xlsx
._outdoors_golden_answers_20210130.csv
outdoors_golden_answers_20210130.csv
outdoors_labels.pickle
outdoors_question_answering_contexts.json
outdoors_questionanswering_test_set.json
outdoors_questionanswering_train_set.json
._posts.csv
posts.csv
predicates.pickle
pull_aips_dependency.py
._question-answer-seed-contexts.csv
question-answer-seed-contexts.csv
question-answer-squad2-guesses.csv
._roberta-base-squad2-outdoors
roberta-base-squad2-outdoors/
roberta-base-squad2-outdoors/._tokenizer_config.json
roberta-base-squad2-outdoors/tokenizer_config.json
roberta-base-squad2-outdoors/._special_tokens_map.json
roberta-base-squad2-outdoors/special_tokens_map.json
roberta-base-squad2-outdoors/._config.json
roberta-base-

## Listing 13.2

### Querying the outdoors dataset with a noun phrase

In [3]:
from aips.data_loaders.outdoors import load_dataframe
#Transform the the outdoors.csv file into solr documents
outdoors_collection = engine.create_collection("outdoors")
outdoors_dataframe = load_dataframe("../../data/outdoors/posts.csv")
outdoors_collection.write(outdoors_dataframe)

outdoors_dataframe.show(5)

Wiping "outdoors" collection
Creating "outdoors" collection
Status: Success
Successfully written 18456 documents
+---+------------------+---------+--------------------+-----+----------+--------------------+-------------+--------------------+--------------------+------------+---------+--------------------+
| id|accepted_answer_id|parent_id|       creation_date|score|view_count|                body|owner_user_id|               title|                tags|answer_count|post_type|                 url|
+---+------------------+---------+--------------------+-----+----------+--------------------+-------------+--------------------+--------------------+------------+---------+--------------------+
|  1|                12|     null|2012-01-24 19:55:...|   31|      7383|A few times I've ...|            9|How do I treat ho...|[health, first ai...|           4| question|https://outdoors....|
|  2|              null|     null|2012-01-24 19:56:...|   31|     10880|In some places, t...|           13|Wher

In [4]:
def get_posts_for_question(query):
    request = {"query": query,
               "limit": 1, 
               "query_fields": ["title", "body"],
               "return_fields": ["id"],
               "filters": [("post_type", "question"), ("accepted_answer_id", "*")]}
    id = outdoors_collection.search(**request)["docs"][0]["id"]
    request = {"query": id,
               "query_fields": ["id", "parent_id"],
               "limit": 3,
               "return_fields": ["id", "post_type", "title", "body", "parent_id", "accepted_answer_id"]}
    return outdoors_collection.search(**request)["docs"]

In [5]:
get_posts_for_question("climbing knots")

[{'id': '18825',
  'accepted_answer_id': 18826,
  'body': "If I wanted to learn how to tie certain knots, or learn about new knots and what they're used for, what are some good resources to look up?",
  'title': "What's a good resource for learning to tie knots for climbing?",
  'post_type': 'question'},
 {'id': '24440',
  'parent_id': 18825,
  'body': "Knots and Ropes for Climbers by Duane Raleigh is a fantastic illustrated resource tailored specifically to climbers. The ABoK is great, but a but beyond the pale of what the average rock climber needs to know. The linked book is a fabulous climber's only reference.",
  'post_type': 'answer'},
 {'id': '18826',
  'parent_id': 18825,
  'body': "Animated Knots By Grog Arguably the best resource online for knot tying is Animated Knots by Grog , it's used by virtually every avid knot tyer I've known. They have excellent step-by-step animations for tying hundreds of common knots for rock climbing, sailing, rescue work, fishing, etc. There are 

In [6]:
def display_questions(query, response, verbose=False):
    print(f"Query {query}:\n")
    if verbose:
        highlights = [response["highlighting"][k] for k in response["highlighting"].keys()]
    print("Ranked Questions:")
    for i, q in enumerate(response["docs"]):
        if verbose:
            print(json.dumps(q, indent="  "))
        if "title" in q.keys():
            id = f'<a href="{q["url"]}">{q["id"]}</a>'
            display(HTML(f'<strong>Question {id}: </strong>{q["title"]}'))
        if verbose:
            display(HTML("<strong>Body:</strong>" + html.unescape(str(highlights[i]["body"][0]))))
            display(HTML("<hr>"))

def search_questions(query, verbose=False):
    request = {"query": query,
               "query_fields": ["title", "body"],
               "limit": 5,
               "return_fields": ["id", "url", "post_type", "title",
                                 "body", "accepted_answer_id"],
               "filters": [("post_type", "question")],
               "hightlight": True}
    response = outdoors_collection.search(**request)
    display_questions(query, response, verbose)

## Listing 13.2

### Querying our collection with a noun phrase

In [7]:
search_questions("climbing knots")

Query climbing knots:

Ranked Questions:


In [8]:
search_questions("snow camping safety")

Query snow camping safety:

Ranked Questions:


## Listing 13.3

### Querying our collection with a question

In [9]:
search_questions("What is DEET?")

Query What is DEET?:

Ranked Questions:


Up next: [Introduction to Transformers](2.introduction-to-transformers.ipynb)